## Model
This notebook will introduce several different methods to build a model using tf.keras.

In [1]:
import tensorflow as tf
import numpy as np 
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
print(tf.__version__)

2.3.0


We will build the same model below with three different methods.
![resources/model.PNG](resources/model.PNG)

<font size="5">The first method </font>: Sequentially add layers to the end of the model.

In [2]:
cnnModel = keras.Sequential()
cnnModel.add(layers.Conv2D(input_shape=[28,28,1],filters = 32, kernel_size = 3, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.Conv2D(filters = 32, kernel_size = 3, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.MaxPool2D(pool_size = 2, strides = 2))
cnnModel.add(layers.Conv2D(filters = 64, kernel_size = 3, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.Conv2D(filters = 64, kernel_size = 3, strides = 1,
                       activation = 'relu'))
cnnModel.add(layers.MaxPool2D(pool_size = 2, strides = 2))
cnnModel.add(layers.Flatten())
cnnModel.add(layers.Dense(units = 512, activation = 'relu'))
cnnModel.add(layers.Dense(units = 10, activation = 'softmax'))
cnnModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0

<font size="5">The second method </font>: Functional API. A layer instance is callable and returns a tensor, representing the current end of the layer. We can call layers multiple times at the same end/tensor to create multiple parallel layers, and call layer with multiple ends/tensors to merge multiple layers.

In [3]:
inputs = tf.keras.Input(shape=(28,28,1))  # Returns a placeholder tensor
x = layers.Conv2D(filters = 32, kernel_size = 3, strides = 1,
                       activation = 'relu')(inputs)
x = layers.Conv2D(filters = 32, kernel_size = 3, strides = 1,
                       activation = 'relu')(x)
x = layers.MaxPool2D(pool_size = 2, strides = 2)(x)
x = layers.Conv2D(filters = 64, kernel_size = 3, strides = 1,
                       activation = 'relu')(x)
x = layers.Conv2D(filters = 64, kernel_size = 3, strides = 1,
                       activation = 'relu')(x)
x = layers.MaxPool2D(pool_size = 2, strides = 2)(x)
x = layers.Flatten()(x)
x = layers.Dense(units = 512, activation = 'relu')(x)
predictions = layers.Dense(units = 10, activation = 'softmax')(x)
cnnModel = tf.keras.Model(inputs=inputs, outputs=predictions)
cnnModel.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 64)         

<font size="5">The third method </font>: We can create a new model by subclassing tf.keras.Model and defining your own forward pass. Create layers in the __init__ method and set them as attributes of the class instance, then define the forward pass in the call method.

In [4]:
class CNNModel(tf.keras.Model):
    def __init__(self):
        super(CNNModel, self).__init__(name='my_model')
        # Define your layers here.
        self.conv1 = layers.Conv2D( filters = 32, kernel_size = 3, strides = 1,
                           activation = 'relu')
        self.conv2 = layers.Conv2D(filters = 32, kernel_size = 3, strides = 1,
                           activation = 'relu')
        self.maxpool1 = layers.MaxPool2D(pool_size = 2, strides = 2)
        self.conv3 = layers.Conv2D(filters = 64, kernel_size = 3, strides = 1,
                           activation = 'relu')
        self.conv4 = layers.Conv2D(filters = 64, kernel_size = 3, strides = 1,
                           activation = 'relu')
        self.maxpool2 = layers.MaxPool2D(pool_size = 2, strides = 2)
        self.dense1 = layers.Dense(units = 512, activation = 'relu')
        self.dense2 = layers.Dense(units = 10, activation = 'softmax')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.maxpool1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.maxpool2(x)
        x = layers.Flatten()(x)
        x = self.dense1(x)
        predictions = self.dense2(x)
        return predictions
cnnModel = CNNModel()
cnnModel.build((10,28,28,1))
cnnModel.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            multiple                  320       
_________________________________________________________________
conv2d_9 (Conv2D)            multiple                  9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_10 (Conv2D)           multiple                  18496     
_________________________________________________________________
conv2d_11 (Conv2D)           multiple                  36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 multiple                  0         
_________________________________________________________________
dense_4 (Dense)              multiple                  524